In [1]:
!conda create --name booknlp python=3.7
!conda activate booknlp

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
| DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/osx-arm64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/osx-arm64/current_repodata.json HTTP/1.1" 304 0
done
Solving environment: unsuccessful attempt using repodata from current_repodata.json, retrying with next repodata source.
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS 

In [2]:
!pip install booknlp
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 11.3 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
from booknlp.booknlp import BookNLP

model_params={"pipeline":"entity,quote,supersense,event,coref", "model":"big"}

booknlp=BookNLP("en", model_params)

using device cpu
{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'big'}
--- startup: 5.479 seconds ---


In [7]:
import os

# Use os.path.expanduser to get the desktop path (works on both Windows and Unix-like systems)
desktop_path = os.path.expanduser("~/Desktop")

# Construct the full path to your file
input_file = os.path.join(desktop_path, '《红楼梦》英文版.txt')

In [8]:
# Output directory to store resulting files in
output_directory="output_dir/红楼梦new/"

# File within this directory will be named ${book_id}.entities, ${book_id}.tokens, etc.
book_id="红楼梦new"

booknlp.process(input_file, output_directory, book_id)

--- spacy: 76.958 seconds ---
--- entities: 442.519 seconds ---
--- quotes: 0.234 seconds ---
--- attribution: 440.532 seconds ---
--- name coref: 2.988 seconds ---
--- coref: 449.400 seconds ---
--- TOTAL (excl. startup): 1414.165 seconds ---, 769634 words


In [17]:
import json
from collections import  Counter

def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data

data=proc("output_dir/红楼梦new/红楼梦new.book")
def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter
class CharQuote:
    def __init__(self,ID,Name,Quote):
        self.ID=ID
        self.Name = Name
        self.Quote = Quote


Namelist = list()
charIDlist = list()
for character in data["characters"]:

    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]

    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        for mention in mentions["proper"]:
            if "Daiyu" in mention["n"]:
                Namelist.append(mention["n"])
                if character_id in charIDlist:
                    pass
                else:
                    charIDlist.append(character_id)

Outputlist = list()
with open("output_dir/红楼梦new/红楼梦new.quotes",'r',encoding='utf-8') as quoteData:
    for line in quoteData:
        cols = line.rstrip().split("\t")
        if cols[5]!="char_id":
            if int(cols[5]) in charIDlist:
                Done = 0
                for name in Namelist:
                    if cols[4] in name and Done == 0:
                        if "Daiyu" in cols[4]:
                            Outputlist.append(CharQuote(cols[5], cols[4], cols[6]))
                            Done = 1
with open("Output.txt",'w',encoding='utf-8') as OutputFile:
    OutputFile.write("CharacterID" +" "+"Name"+" "+"Quote")
    for item in Outputlist:
        OutputFile.write(item.ID+" "+item.Name+" "+item.Quote+"\n")

In [16]:
import re

with open('《红楼梦》英文版.txt', 'r', encoding='utf-8') as input_file:
    content = input_file.read()

dialogs = re.findall(r'"(.*?)"', content)

with open('完整对话.txt', 'w', encoding='utf-8') as output_file:
    for dialog in dialogs:
        output_file.write(dialog + '\n')